In [1]:
%load_ext autoreload
%autoreload 2

# Добавляем путь к основной папке проекта, чтобы иметь возможность делать импорт из src
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [7]:
import pickle
import tqdm

import pandas as pd

from censorability_monitor.mempool_analysis import get_transaction_status, divide_mempool_to_three_parts

In [3]:
tr_data = pd.read_csv('../data/merged_221220_221231_with_contracts.csv')
tr_data.head()

,tx_hash,from,to,gas,gasPrice,blockNumber,timestamp_x,miner,timestamp_y,waiting,...,is_contract_transaction,baseFeePerGas,wait_time_adjusted,is_ofac,priority_fee,maxFeePerGas,maxPriorityFeePerGas,gasUsed,effectiveGasPrice,nonce
0,0x14e970dba71ef93b42f337d3494c4d6e8fee4d83ce38...,0x498B859D2e59958E209d7DD262324C8D31b12B12,0xC1E088fC1323b20BCBee9bd1B9fC9546db5624C5,242494,117.114005,16221291,1671483611,0x690B9A9E9aa1C9dB991C7721a92d351Db4FaC990,NaN,NaN,...,True,17.248596,0.0,False,99.865409,117.114005,117.114005,189564,117.114005,12269
1,0x33fcc824c05553a3ec31a7d71dc6dbb26cf51d608009...,0x07B873DE48807ec7B02B38cc31b4f96a6A507182,0x883B01205b938EC7268286d13Cb40a2045D2E78F,800000,23.932889,16221291,1671483611,0x690B9A9E9aa1C9dB991C7721a92d351Db4FaC990,NaN,NaN,...,True,17.248596,0.0,False,6.684293,23.932889,23.932889,214272,23.932889,907
2,0x555ffb08812804e23fe18467886edaa167f502488c6b...,0xAC813c6b767F3C2f1764994F9D3E4D38A8df6F32,0x0b382C0A00D6080b0009F003C050eAC3c30f0000,305170,17.248596,16221291,1671483611,0x690B9A9E9aa1C9dB991C7721a92d351Db4FaC990,NaN,NaN,...,True,17.248596,0.0,False,0.000000,17.248596,17.248596,152585,17.248596,1689
3,0x9febfdc63677cd0decb532cb049004add2959d7f1a1f...,0xf7C8133baEa3313993B3c476c016aa869C991679,0x91c8D7e4080bed28e26FCE1b87cACcFaF7bbF794,292432,18.212433,16221291,1671483611,0x690B9A9E9aa1C9dB991C7721a92d351Db4FaC990,1.671484e+09,3.0,...,True,17.248596,3.0,False,0.963838,0.000000,0.000000,208202,18.212433,19
4,0xe24c2fb8e8d581d86710a0ca685e27ed6c36ca722d8e...,0xAC813c6b767F3C2f1764994F9D3E4D38A8df6F32,0x0b382C0A00D6080b0009F003C050eAC3c30f0000,245026,20.798797,16221291,1671483611,0x690B9A9E9aa1C9dB991C7721a92d351Db4FaC990,NaN,NaN,...,True,17.248596,0.0,False,3.550201,20.798797,20.798797,122513,20.798797,1690


In [4]:
tr_data.loc[tr_data['maxFeePerGas'] == 0, 'maxFeePerGas'] = tr_data.loc[tr_data['maxFeePerGas'] == 0, 'gasPrice']
tr_data.loc[tr_data['maxPriorityFeePerGas'] == 0, 'maxPriorityFeePerGas'] = tr_data.loc[tr_data['maxPriorityFeePerGas'] == 0, 'maxFeePerGas']

In [5]:
tr_data['first_seen'] = tr_data['timestamp_y']
tr_data.loc[tr_data['timestamp_y'].isna(), 'first_seen'] = tr_data.loc[tr_data['timestamp_y'].isna(), 'timestamp_x'] - 1

In [6]:
import web3
from web3 import Web3, HTTPProvider, IPCProvider 

geth_ipc = '/media/Warehouse/Warehouse/Ethereum/data/.ethereum/geth.ipc'
w3 = Web3(IPCProvider(geth_ipc))
w3_drpc = Web3(HTTPProvider('https://main.drpc.org/ogrpc?network=ethereum&dkey=AhVPNnFg1020t9FEr-MKVyPunnyCo98R7bAt5s-wSZOx'))
print ("Latest Ethereum block number DRPC" , w3_drpc.eth.blockNumber)
print ("Latest Ethereum block number" , w3.eth.blockNumber)

Latest Ethereum block number DRPC 16564153
Latest Ethereum block number 16564153


In [8]:
ofac_txs = tr_data[tr_data['is_ofac'] == True].copy()
# Соберем номера блоков, которые нам интересны (должны содержать в мемпуле ofac транзакции)
ofac_txs.head()
interesting_blocks = set()

for i, row in ofac_txs.iterrows():
    block_with_ofac = row['blockNumber']
    waiting_time = row['wait_time_adjusted']
    n_blocks_before = int(waiting_time) // 12 + 1
    new_interesting_blocks = list(range(block_with_ofac, block_with_ofac - n_blocks_before - 1, -1))
    interesting_blocks.update(new_interesting_blocks)

interesting_blocks = sorted(interesting_blocks)

In [9]:
first_block = tr_data['blockNumber'].min()
first_20_blocks = list(range(first_block + 1, first_block + 21))

interesting_blocks = set(interesting_blocks)
interesting_blocks.update(first_20_blocks)
interesting_blocks = sorted(interesting_blocks)

In [10]:
with open('../data/twelve_days_blocks.pickle', 'rb') as f:
    blocks_info = pickle.load(f)

In [ ]:
for b_number in tqdm.tqdm([n for n in interesting_blocks if n >= 16253260]):
    block_ts = blocks_info[b_number]['timestamp']
    baseFeePerGas = blocks_info[b_number]['baseFeePerGas'] / 10 ** 9

    current_block_tx, next_block_tx, after_next_block_tx =  divide_mempool_to_three_parts(
        b_number, tr_data, w3)

    not_included = pd.concat([current_block_tx, next_block_tx, after_next_block_tx])
    addresses = not_included['from'].unique()

    # Fetch and save account details
    account_details = {}

    for addr in addresses:
        fetched = False
        while not fetched:
            try:
                account_balance = w3_drpc.eth.get_balance(addr, b_number - 1) / 10 ** 18
                n_account_transactions = w3_drpc.eth.get_transaction_count(addr, b_number - 1)
                fetched = True
            except Exception as e:
                print(f'Error {b_number}  {e}, trying again...') 
        account_details[addr] = {
            'tx_count': n_account_transactions,
            'account_balance': account_balance
        }
    with open(f'../block_addresses_details/block_{b_number}_address_details.pickle', 'wb') as f:
        pickle.dump(account_details, f)

  0%|                                                                                        | 0/6594 [00:00<?, ?it/s]

Error 16253260  {'message': 'No response or no available upstream for eth_getBalance', 'code': 1}, trying again...
Error 16253260  {'message': 'No response or no available upstream for eth_getBalance', 'code': 1}, trying again...
Error 16253260  {'message': 'No response or no available upstream for eth_getBalance', 'code': 1}, trying again...
Error 16253260  {'message': 'No response or no available upstream for eth_getBalance', 'code': 1}, trying again...
Error 16253260  {'message': 'No response or no available upstream for eth_getBalance', 'code': 1}, trying again...
Error 16253260  {'message': 'No response or no available upstream for eth_getBalance', 'code': 1}, trying again...
Error 16253260  {'message': 'No response or no available upstream for eth_getBalance', 'code': 1}, trying again...
Error 16253260  {'message': 'No response or no available upstream for eth_getBalance', 'code': 1}, trying again...
Error 16253260  {'message': 'No response or no available upstream for eth_getTra

  1%|▍                                                                      | 36/6594 [10:08:08<337:39:04, 185.35s/it]

Error 16253853  {'message': 'No response or no available upstream for eth_getBalance', 'code': 1}, trying again...
Error 16253853  {'message': 'No response or no available upstream for eth_getBalance', 'code': 1}, trying again...
Error 16253853  {'message': 'No response or no available upstream for eth_getBalance', 'code': 1}, trying again...


  3%|██▏                                                                   | 208/6594 [17:58:38<319:11:40, 179.94s/it]

...

  7%|█████▏                                                                | 490/6594 [73:48:45<297:53:07, 175.69s/it]

Error 16262417  400 Client Error: Bad Request for url: https://main.drpc.org/ogrpc?network=ethereum&dkey=AhVPNnFg1020t9FEr-MKVyPunnyCo98R7bAt5s-wSZOx, trying again...
Error 16262417  400 Client Error: Bad Request for url: https://main.drpc.org/ogrpc?network=ethereum&dkey=AhVPNnFg1020t9FEr-MKVyPunnyCo98R7bAt5s-wSZOx, trying again...
Error 16262417  400 Client Error: Bad Request for url: https://main.drpc.org/ogrpc?network=ethereum&dkey=AhVPNnFg1020t9FEr-MKVyPunnyCo98R7bAt5s-wSZOx, trying again...
Error 16262417  400 Client Error: Bad Request for url: https://main.drpc.org/ogrpc?network=ethereum&dkey=AhVPNnFg1020t9FEr-MKVyPunnyCo98R7bAt5s-wSZOx, trying again...
Error 16262417  400 Client Error: Bad Request for url: https://main.drpc.org/ogrpc?network=ethereum&dkey=AhVPNnFg1020t9FEr-MKVyPunnyCo98R7bAt5s-wSZOx, trying again...
Error 16262417  400 Client Error: Bad Request for url: https://main.drpc.org/ogrpc?network=ethereum&dkey=AhVPNnFg1020t9FEr-MKVyPunnyCo98R7bAt5s-wSZOx, trying again..

  7%|█████▏                                                                | 493/6594 [73:57:21<293:57:11, 173.45s/it]

Error 16262518  {'message': 'No state available for block 0xf8ad4c85990b77947a48cf0b91e6c6eff8183fcd0e11af10e0f500dc63b47ed8', 'code': 1}, trying again...


  9%|██████▏                                                               | 586/6594 [84:23:35<243:07:17, 145.68s/it]